In [1]:
import os
import tensorflow as tf

import ingestor
import extractor2
import utils

1.14.0


In [2]:
input_fn, params = ingestor.get_input_fn()







INFO:tensorflow:PASSED IN parser is None


In [3]:
features, labels = input_fn()[0], input_fn()[1]

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [4]:
dataset_dir = params['dataset_dir']
goal_file = os.path.join(dataset_dir, params['goal_vocab'])
goal_table = utils.vocab_table_from_file(goal_file)
print('goal_table')
print(goal_table)
thms_file = os.path.join(dataset_dir, params['thm_vocab'])
thms_table = utils.vocab_table_from_file(thms_file)
print('thms_table')
print(thms_table)
# check if there is a significant difference between static and hash table


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

goal_table
thms_table


In [5]:
features

{'goal': <tf.Tensor: id=234, shape=(4,), dtype=string, numpy=
 array([b"(a (c (fun (fun (fun (cart (real) N) (bool)) (bool)) (bool)) ?) (l (v (fun (cart (real) N) (bool)) u2') (a (a (c (fun (bool) (fun (bool) (bool))) /\\) (a (a (c (fun (fun (cart (real) (finite_sum N (1))) (cart (real) N)) (fun (fun (cart (real) (finite_sum N (1))) (bool)) (bool))) continuous_on) (v (fun (cart (real) (finite_sum N (1))) (cart (real) N)) f)) (v (fun (cart (real) (finite_sum N (1))) (bool)) t))) (a (a (c (fun (bool) (fun (bool) (bool))) /\\) (a (a (c (fun (fun (cart (real) N) (bool)) (fun (fun (cart (real) N) (bool)) (bool))) SUBSET) (a (a (c (fun (fun (cart (real) (finite_sum N (1))) (cart (real) N)) (fun (fun (cart (real) (finite_sum N (1))) (bool)) (fun (cart (real) N) (bool)))) IMAGE) (v (fun (cart (real) (finite_sum N (1))) (cart (real) N)) f)) (v (fun (cart (real) (finite_sum N (1))) (bool)) t))) (a (a (c (fun (fun (cart (real) N) (bool)) (fun (fun (cart (real) N) (bool)) (fun (cart (real) N) (boo

In [6]:
labels

{'tac_id': <tf.Tensor: id=352, shape=(4,), dtype=int64, numpy=array([ 3,  5, 15, 15])>}

In [7]:
features, labels = extractor2.Extractor(params).extractor(features, labels)

Instructions for updating:
Use `tf.cast` instead.


In [8]:
features

{'goal_ids': <tf.Tensor: id=418, shape=(4, 1000), dtype=int32, numpy=
 array([[ 2,  6,  9, ...,  7,  5, 45],
        [ 2,  6,  6, ...,  0,  0,  0],
        [ 2,  6,  6, ...,  0,  0,  0],
        [ 2,  6,  6, ...,  0,  0,  0]], dtype=int32)>,
 'thm_ids': <tf.Tensor: id=444, shape=(4, 137), dtype=int32, numpy=
 array([[ 2,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 2,  6,  8,  4,  4,  7,  5,  5, 19, 15, 10,  7, 16,

In [9]:
labels

{'tac_id': <tf.Tensor: id=352, shape=(4,), dtype=int64, numpy=array([ 3,  5, 15, 15])>}